In [37]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [38]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = np.expand_dims(train_images, axis=-1)
train_images = np.repeat(train_images, 3, axis=-1)
train_images = np.asarray([img_to_array(array_to_img(img, scale=False).resize((48, 48))) for img in train_images])

test_images = np.expand_dims(test_images, axis=-1)
test_images = np.repeat(test_images, 3, axis=-1)  # Повторення каналів для отримання RGB
test_images = np.asarray([img_to_array(array_to_img(img, scale=False).resize((48, 48))) for img in test_images])

train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
train_features = vgg16_base.predict(train_images)
test_features = vgg16_base.predict(test_images)

train_features = np.reshape(train_features, (train_features.shape[0], -1))
test_features = np.reshape(test_features, (test_features.shape[0], -1))

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
test_labels_categorical = to_categorical(test_labels)

313/313 [==============================] - 74s 237ms/step


In [39]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=train_features.shape[1]))
model.add(Dense(10, activation='softmax'))  # Змініть кількість нейронів в останньому шарі на кількість класів

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [40]:
model.fit(train_features, train_labels, epochs=50, batch_size=160, validation_data=(test_features, test_labels))

Epoch 1/50
375/375 [==============================] - 2s 5ms/step - loss: 0.5563 - accuracy: 0.8118 - val_loss: 0.4325 - val_accuracy: 0.8468
Epoch 2/50
375/375 [==============================] - 2s 5ms/step - loss: 0.3937 - accuracy: 0.8570 - val_loss: 0.3962 - val_accuracy: 0.8593
Epoch 3/50
375/375 [==============================] - 2s 5ms/step - loss: 0.3616 - accuracy: 0.8690 - val_loss: 0.3872 - val_accuracy: 0.8615
Epoch 4/50
375/375 [==============================] - 2s 5ms/step - loss: 0.3413 - accuracy: 0.8759 - val_loss: 0.3675 - val_accuracy: 0.8691
Epoch 5/50
375/375 [==============================] - 2s 5ms/step - loss: 0.3237 - accuracy: 0.8830 - val_loss: 0.3650 - val_accuracy: 0.8685
Epoch 6/50
375/375 [==============================] - 2s 5ms/step - loss: 0.3106 - accuracy: 0.8863 - val_loss: 0.3620 - val_accuracy: 0.8701
Epoch 7/50
375/375 [==============================] - 2s 5ms/step - loss: 0.3021 - accuracy: 0.8895 - val_loss: 0.3508 - val_accuracy: 0.8748
Epoch 

In [42]:
predictions = model.predict(test_features)
test_loss, test_accuracy = model.evaluate(test_features, test_labels)
print(f'Точність алгоритму: {np.round(test_accuracy * 100, 2)}%')

313/313 [==============================] - 0s 1ms/step - loss: 0.4210 - accuracy: 0.8884
Точність алгоритму: 88.84%
